In [40]:
import pandas as pd
import matplotlib.pyplot as plt

df : pd.DataFrame = pd.read_csv("../data/credit_transactions.csv", dtype={'merchant_zip':str})

df.columns
df.head()

,user_id,year,month,day,time,amount,card_method,merchant,merchant_city,merchant_state,...,user_apartment,user_city,user_state,user_zip,user_lat,user_long,user_income,user_debt,user_rating,user_number_of_cards
0,0,2015,11,15,12:55,$287.13,Online Transaction,-8194607650924472520,ONLINE,NaN,...,NaN,La Verne,CA,91750,34.15,-117.76,$59696,$127613,787,5
1,0,2015,11,15,13:19,$2.41,Online Transaction,-7759074308363763111,ONLINE,NaN,...,NaN,La Verne,CA,91750,34.15,-117.76,$59696,$127613,787,5
2,0,2015,11,16,09:41,$50.81,Online Transaction,-551332107213382088,ONLINE,NaN,...,NaN,La Verne,CA,91750,34.15,-117.76,$59696,$127613,787,5
3,0,2015,11,16,09:46,$248.36,Online Transaction,4872340518840476610,ONLINE,NaN,...,NaN,La Verne,CA,91750,34.15,-117.76,$59696,$127613,787,5
4,0,2015,11,16,11:20,$473.00,Online Transaction,-8566951830324093739,ONLINE,NaN,...,NaN,La Verne,CA,91750,34.15,-117.76,$59696,$127613,787,5


# Dropping Useless Columns

In [37]:
df.drop(columns=['merchant'], inplace=True)

# Fixing Easy Dtypes

In [41]:
df['merchant_zip']

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
          ...   
99752    91502.0
99753    22079.0
99754        NaN
99755    95111.0
99756    11563.0
Name: merchant_zip, Length: 99757, dtype: object

# Condensing Transaction Datetime

In [31]:
from datetime import datetime

def condense_time(row):
    x = datetime(
        int(row.year), 
        int(row.month), 
        int(row.day), 
        int(row.time.split(':')[0]), 
        int(row.time.split(':')[1])
    )
    return x

df['purchase_date'] = df.apply(condense_time, axis=1)

df.drop(columns=['year','month','day','time'], inplace=True)

# Fixing Dollar Amounts

In [32]:
df['amount'] = df['amount'].str.replace("$", "").astype(float)

# Adding External MCC Category Names

In [33]:
mcc_df = pd.read_csv('../data/mcc_grouped.csv')

def get_mcc_name(code):
    matching_code = mcc_df.loc[(mcc_df['start'] <= code) & (code <= mcc_df['end'])]
    if not matching_code.empty:
        return matching_code['Business type'].iloc[0]
    else:
        return 'Unknown'
    
# Apply the custom function to create a new column in df
df['mcc_group'] = df['merchant_code'].apply(get_mcc_name)

df.drop(
    columns=['merchant_code'], 
    errors='ignore',
    inplace=True
)

In [34]:
df.to_csv('../data/cleaned.csv')